# Numpy Compat

In [1]:
from uarray import *
import numpy as np
from numba import njit

In [2]:
args = [np.arange(1000), np.arange(10)]

In [3]:
@optimize
def some_fn(a, b):
    return np.multiply.outer(a, b)[5]

In [15]:
some_fn.__optimize_steps__['resulting_expr']

Index(Sequence(Value('1'), VectorCallable(Content(Scalar(Value('5'))))),
      OuterProduct(Function(Content(Call(Ufunc(<ufunc 'multiply'>),
                                         Scalar(Unbound('', variable_name=i0)),
                                         Scalar(Unbound('', variable_name=i1)))),
                            Unbound('', variable_name=i0),
                            Unbound('', variable_name=i1)),
                   NPArray(Expression(Name(id='a', ctx=Load()))),
                   NPArray(Expression(Name(id='b', ctx=Load())))))

In [4]:
print(some_fn.__optimize_steps__['ast_as_source'])



def fn(a, b):
    i_1 = 5
    i_3 = a
    i_4 = b
    i_2 = np.multiply.outer(i_3, i_4)
    i_0 = i_2[i_1]
    return i_0



In [5]:
some_fn(*args)

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

In [6]:
@optimize
def some_fn_t(a, b):
    a = a.has_dim(1)
    b = b.has_dim(1)
    return np.multiply.outer(a, b)[5]

In [13]:
some_fn_t.__optimize_steps__['resulting_expr']

Index(Sequence(Value('1'), VectorCallable(Content(Scalar(Value('5'))))),
      OuterProduct(Function(Content(Call(Ufunc(<ufunc 'multiply'>),
                                         Scalar(Unbound('', variable_name=i2)),
                                         Scalar(Unbound('', variable_name=i3)))),
                            Unbound('', variable_name=i2),
                            Unbound('', variable_name=i3)),
                   ToSequenceWithDim(NPArray(Expression(Name(id='a', ctx=Load()))),
                                     Value('1')),
                   ToSequenceWithDim(NPArray(Expression(Name(id='b', ctx=Load()))),
                                     Value('1'))))

In [7]:
print(some_fn_t.__optimize_steps__['ast_as_source'])



def fn(a, b):
    i_10 = 1
    i_6 = ([0] * i_10)
    for i_11 in range(i_10):
        i_12 = (b.shape[0],)[i_11]
        i_6[i_11] = i_12
    i_5 = np.empty(tuple(i_6))
    i_7 = b.shape[0]
    for i_8 in range(i_7):
        i_15 = 5
        i_16 = a
        i_13 = i_16[i_15]
        i_17 = i_8
        i_18 = b
        i_14 = i_18[i_17]
        i_9 = (i_13 * i_14)
        i_5[i_8] = i_9
    return i_5



In [8]:
some_fn_t(np.arange(10), np.arange(10))

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45.])

In [9]:
%time some_fn(*args)

CPU times: user 256 µs, sys: 117 µs, total: 373 µs
Wall time: 309 µs


array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

In [10]:
%time some_fn_t(*args)

CPU times: user 28 µs, sys: 1e+03 ns, total: 29 µs
Wall time: 34.1 µs


array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45.])

In [11]:
numba_optimized = numba.njit

NameError: name 'numba' is not defined